In [1]:
print(sc)
print("Spark version: {a}".format(a=sc.version))
print(spark)

<SparkContext master=local[*] appName=PySparkShell>
Spark version: 3.1.2


In [2]:
from pyspark.ml.regression import LinearRegression
#from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.datasets import load_diabetes
import pandas
from pyspark.ml.feature import VectorAssembler

In [3]:
diabetes=load_diabetes()
datos = pandas.DataFrame(diabetes.data, columns=diabetes.feature_names)
datos['target'] = pandas.Series(diabetes.target)
datos=spark.createDataFrame(datos)
datos.show(n=10,truncate=8,vertical=False)

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------+
|     age|     sex|     bmi|      bp|      s1|      s2|      s3|      s4|      s5|      s6|target|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------+
|0.038...|0.050...|0.061...|0.021...|-0.04...|-0.03...|-0.04...|-0.00...|0.019...|-0.01...| 151.0|
|-0.00...|-0.04...|-0.05...|-0.02...|-0.00...|-0.01...|0.074...|-0.03...|-0.06...|-0.09...|  75.0|
|0.085...|0.050...|0.044...|-0.00...|-0.04...|-0.03...|-0.03...|-0.00...|0.002...|-0.02...| 141.0|
|-0.08...|-0.04...|-0.01...|-0.03...|0.012...|0.024...|-0.03...|0.034...|0.022...|-0.00...| 206.0|
|0.005...|-0.04...|-0.03...|0.021...|0.003...|0.015...|0.008...|-0.00...|-0.03...|-0.04...| 135.0|
|-0.09...|-0.04...|-0.04...|-0.01...|-0.06...|-0.07...|0.041...|-0.07...|-0.04...|-0.09...|  97.0|
|-0.04...|0.050...|-0.04...|-0.01...|-0.04...|-0.02...|7.788...|-0.03...|-0.06...|-0.03...| 138.0|
|0.063...|

In [10]:
def prepare_data(df,xs,print_outcome=True):
    vAssembler = VectorAssembler(inputCols=xs,outputCol='features')
    vdatos = vAssembler.transform(df).select(['features', 'target'])
    traindata, testdata = vdatos.randomSplit(weights=[0.7, 0.3],seed=123)
    if print_outcome==True:
        traindata.show(n=5,truncate=50,vertical=False)
        testdata.show(n=5,truncate=50,vertical=False)
    return(dict(train_set=traindata, test_set=testdata))

def do_model(df_train,print_outcome=True):
    lr = LinearRegression(
        featuresCol='features', labelCol='target', predictionCol='prediction',
        regParam=0.0, #no regularization
        fitIntercept=True,
        standardization=False,
        loss='squaredError')
    lr_model = lr.fit(df_train)
    if print_outcome==True:
        print("Adjusted R squared: {a} \n".format(a=lr_model.summary.r2adj))
        print("RMSE: {a}".format(a=lr_model.summary.rootMeanSquaredError))
    return(lr_model)

In [11]:
splitted_data=prepare_data(df=datos, xs=list(datos.columns[0:-1]))

modelo=do_model(df_train=splitted_data['train_set'])

datos.select('target').describe().show()

+--------------------------------------------------+------+
|                                          features|target|
+--------------------------------------------------+------+
|[-0.107225631607358,-0.044641636506989,-0.07734...| 137.0|
|[-0.103593093156339,-0.044641636506989,-0.03746...| 113.0|
|[-0.0963280162542995,-0.044641636506989,-0.0838...| 101.0|
|[-0.0963280162542995,-0.044641636506989,-0.0762...| 134.0|
|[-0.0963280162542995,-0.044641636506989,-0.0697...| 158.0|
+--------------------------------------------------+------+
only showing top 5 rows

+--------------------------------------------------+------+
|                                          features|target|
+--------------------------------------------------+------+
|[-0.099960554705319,-0.044641636506989,-0.06764...|  55.0|
|[-0.0963280162542995,-0.044641636506989,-0.0363...| 200.0|
|[-0.0926954778032799,-0.044641636506989,0.02828...| 302.0|
|[-0.0890629393522603,-0.044641636506989,-0.0115...| 206.0|
|[-0.0854304009

In [6]:
def show_pvalues(trained_model,xs):
    estimate=list(trained_model.coefficients)
    estimate.append(trained_model.intercept)
    nombres=xs
    nombres.append("intercept")
    outcome=pandas.DataFrame(estimate,columns=["estimate"],index=nombres)
    outcome["p values"]=list(trained_model.summary.pValues)
    return(outcome)

results=show_pvalues(trained_model=modelo, xs=list(datos.columns[0:-1]))
results

,estimate,p values
age,-8.692342,9.027008e-01
sex,-223.883324,2.849621e-03
bmi,513.054880,5.714629e-11
bp,278.001549,4.374466e-04
s1,-1090.910707,3.132334e-02
s2,651.827811,1.171414e-01
s3,301.628362,2.314252e-01
s4,443.292881,2.053897e-02
s5,779.027175,1.869308e-04
s6,77.692155,3.148062e-01
